# Basic examples

## import dependencies

In [17]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Ajuste selon la structure du projet
sys.path.append(project_root)

from src.surreal_orm import (
    BaseSurrealModel,
    SurrealDBConnectionManager,
    SurrealConfigDict,
)
from dotenv import load_dotenv

### Setup Database Connexion config

You have to set a SurrealDB in your environnement, docker or service.


In [18]:
# Load environnement from .env (copy and rename .env.example in needed)
load_dotenv()

SURREALDB_URL = os.getenv("SURREALDB_URL") or ""
SURREALDB_USER = os.getenv("SURREALDB_USER") or ""
SURREALDB_PASS = os.getenv("SURREALDB_PASS") or ""
SURREALDB_NAMESPACE = os.getenv("SURREALDB_NAMESPACE") or ""
SURREALDB_DATABASE = os.getenv("SURREALDB_DATABASE") or ""


SurrealDBConnectionManager.set_connection(
    SURREALDB_URL,
    SURREALDB_USER,
    SURREALDB_PASS,
    SURREALDB_NAMESPACE,
    SURREALDB_DATABASE,
)

### Create the exemples model

In [19]:
class User(BaseSurrealModel):
    model_config = SurrealConfigDict(primary_key="email")
    name: str
    age: int
    email: str

### Basic examples

#### Create Users

If you want to recreate user you need to delete it first.


In [20]:
user = User(name="John Doe", age=30, email="john.doe@example.com")
await user.save()

print("User created:", user)

user2 = User(name="Jeanne Doe", age=28, email="jeanne.doe@example.com")
await user2.save()

print("User2 created:", user2)

User created: name='John Doe' age=30 email='john.doe@example.com'
User2 created: name='Jeanne Doe' age=28 email='jeanne.doe@example.com'


#### Select all users

In [21]:
users = await User.objects().all()

for user_item in users:
    print(user_item.get_id())

jeanne.doe@example.com
john.doe@example.com


#### Select the first on a filter by name

In [22]:
user = await User.objects().filter(name="John Doe").first()
print(user)

name='John Doe' age=30 email='john.doe@example.com'


#### Select somes field only. Retrives dict of selected fields instead of object.

In [23]:
users = await User.objects().select("name", "age").exec()
for user_item in users:
    print(user_item)

{'age': 28, 'name': 'Jeanne Doe'}
{'age': 30, 'name': 'John Doe'}


#### Update user (update all fields)

In [24]:
user = await User.objects().filter(name="John Doe").first()
user.age = 31
await user.update()
print("User updated:", user)

User updated: name='John Doe' age=31 email='john.doe@example.com'


#### Merge user (change only the selected field of the user)

In [25]:
user = await User.objects().get("john.doe@example.com")
print(user)
await user.merge(age=32)
print("User merged:", user)

name='John Doe' age=31 email='john.doe@example.com'
User merged: name='John Doe' age=31 email='john.doe@example.com'


#### Delete a selected user

In [26]:
user = await User.objects().filter(name="John Doe").first()
await user.delete()
print("User deleted:", user)

User deleted: name='John Doe' age=32 email='john.doe@example.com'


#### Django look DoesNotExit

In [27]:
# If user doesn't exit
try:
    user = await User.objects().get("john.smith@@example.com")
    user.age = 41
    await user.update()
    print("User exist")
except User.DoesNotExist:
    user = User(name="John Smith", age=41, email="john.smith@@example.com")
    await user.save()
    print("User does not exist")

User does not exist


In [28]:
# If user Exit
try:
    user = await User.objects().get("john.smith@@example.com")
    user.age = 42
    await user.update()
    print("User exist")
except User.DoesNotExist:
    user = User(name="John Smith", age=42, email="john.smith@@example.com")
    await user.save()
    print("User does not exist")

User exist


#### Delete all record in the table. (don't delete the table itself)

In [29]:
await User.objects().delete_table()

True

- Fin